In [70]:
import pickle

import numpy as np
import struct
import math
import os

In [71]:
# 加载minist数据集
# 加载数据集这方面的工作参考了前人的工作，网页为：https://blog.csdn.net/hxxjxw/article/details/113727973
def load_mnist(mnistdir , train):
    ministfile = open(mnistdir,'rb')
    ministdata = ministfile.read()
    ministfile.close()
    rows=1
    cols=1
    # 加载训练集
    if train:
        # 解析文件头信息，依次为魔数、图片数量、每张图片高、每张图片宽
        # 因为数据结构中前4行的数据类型都是32位整型，所以采用i格式，但我们需要读取前4行数据，所以需要4个i。我们后面会看到标签集中，只使用2个ii
        magic_num,images,rows,cols = struct.unpack_from('>iiii', ministdata,0)
    else:
        # 加载标签集
        magic_num,images = struct.unpack_from('>ii', ministdata,0)
    print('魔数:%d, 图片数量: %d张, 图片大小: %d*%d' % (magic_num, images, rows, cols))
    # 计算加载的总像素是多少
    size = images * rows * cols
    # calcsize获得数据在缓存中的指针位置，从前面介绍的数据结构可以看出，读取了前4行之后，指针位置（即偏移位置offset）指向0016
    if train:
        pointer = struct.calcsize('>iiii')
    else :
        pointer =  struct.calcsize('>ii')
    pack_data = struct.unpack_from('>' + str(size) + 'B', ministdata,pointer)
    if train:
        pack_data = np.reshape(pack_data,[images,rows,cols])
    else:
        pack_data = np.reshape(pack_data,[images])
    # 最终返回了一个矩阵，矩阵的大小由是训练集还是标签集决定
    # 训练集就相当于返回好多页纸，每一页纸上面有对应的行数和列数
    print('本次解析的矩阵格式为[%d,%d,%d]' % (images,rows,cols))
    return pack_data

In [72]:
# 在本函数中完成了本地的数据加载
# 根据参数取出相应的验证集并做正则化
def load_minist_data():
    trainImages = load_mnist("data/train-images.idx3-ubyte",True)
    trainLabels = load_mnist("data/train-labels.idx1-ubyte",False)
    testImages = load_mnist("data/t10k-images.idx3-ubyte",True)
    testLabels = load_mnist("data/t10k-labels.idx1-ubyte",False)
    # 其实这里不知道为什么要pad，但还是先pad一下
    # 我觉得可能是因为卷积的时候不丢失信息吧
    # 对矩阵进行进行填充
    # https://blog.csdn.net/qq_34650787/article/details/80500407
    trainImages = np.pad(trainImages, ((0, 0), (2, 2), (2, 2)))
    testImages = np.pad(testImages, ((0, 0), (2, 2), (2, 2)))

    # 获取矩阵维度
    real_num, real_rows, real_cols = trainImages.shape
    real_num2,real_rows2,real_cols2 = testImages.shape
    # print('real_rows=%d,real_cols=%d' % (real_rows,real_cols))

    # 这个类型转换我也搞不太懂
    # 而且也不知道这个reshape的目的在哪里
    # 进行类型转换 https://blog.csdn.net/u012267725/article/details/77489244
    # 我的理解是在这里把很多页书拼起来拼成一页，但是这一页很长
    # print(trainImages.shape) #:这里他的输出是（60000,32,32）
    trainImages = trainImages.astype(np.float32).reshape(real_num, 1, real_rows, real_cols)
    testImages = testImages.astype(np.float32).reshape(real_num2, 1, real_rows, real_cols)
    # print(trainImages.shape) #:在这里输出就变成了（60000,1,32,32）

    # 接下来在训练集中划分验证集
    # 这里的参数是可以调整的
    varProof = -500 # 取最后的500个
    proofImages = trainImages[varProof:0]
    proofLabels = trainLabels[varProof:0]
    trainImages = trainImages[0:varProof]
    trainLabels = trainLabels[0:varProof]

    # 对数据进行归一化，这里其实也可以做一个参数选项，可选可不选
    # https://blog.csdn.net/sdgfbhgfj/article/details/123780347
    if True:
        mean = np.mean(trainImages,axis=0)
    else:
        mean = np.zeros_like(trainImages)
    trainImages -= mean
    proofImages -= mean
    testImages -= mean
    print('加载完毕')
    # 依次返回训练数据&标签 验证数据&标签 测试数据&标签
    return trainImages,trainLabels,proofImages,proofLabels,testImages,testLabels

In [73]:
# 用来验证数据加载的正确性
load_minist_data()

魔数:2051, 图片数量: 60000张, 图片大小: 28*28
本次解析的矩阵格式为[60000,28,28]
魔数:2049, 图片数量: 60000张, 图片大小: 1*1
本次解析的矩阵格式为[60000,1,1]
魔数:2051, 图片数量: 10000张, 图片大小: 28*28
本次解析的矩阵格式为[10000,28,28]
魔数:2049, 图片数量: 10000张, 图片大小: 1*1
本次解析的矩阵格式为[10000,1,1]
(60000, 32, 32)
(60000, 1, 32, 32)
加载完毕


(array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],
 
 
        [[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],
 
 
        [[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],
 
 
        ...,
 
 
        [[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
   

In [ ]:
# 在这个cell实现随机梯度下降
"""
随机梯度下降算法（Stochastic gradient descent，SGD）在神经网络模型训练中，是一种很常见的优化算法。
这个算法的流程就是在每次更新的时候使用一个样本进行梯度下降，所谓的随机二字，就是说我们可以随机用一个样本来表示所有的样本，来调整超参数θ
程序中的参数命名来源于下面的博客：
https://blog.csdn.net/Oscar6280868/article/details/90641638
"""
def sgd(theta,xj,parameters):
    if parameters is None:
        # 创建学习率参数
        parameters = {'learn_rate': 1e-2}
    theta = theta - parameters['learn_rate'] * xj
    return theta,parameters

In [ ]:
# 实现一个自适应动量的随机优化方法
# 我觉得到时候看看能不能把这个删掉...太明显了
"""
论文：https://arxiv.org/pdf/1412.6980.pdf
参数说明：
t t：更新的步数（steps）
α lphaα：学习率，用于控制步幅（stepsize）
θ hetaθ：要求解（更新）的参数
β1 一阶矩衰减系数
β2 二阶矩衰减系数
f（θ） 目标函数
g 目标函数对θ求导所得梯度
m 梯度g的一阶矩
v 梯度g的二阶矩

参考的博客：https://blog.csdn.net/sinat_36618660/article/details/100026261
"""
def adam(theta,xj,parameters):
    if parameters is None:
        parameters = {
            "learn_rate":1e-2,
            "β1":0.9,
            "β2":0.9,
            "e":1e-9,
            "m":np.zeros_like(theta),
            "v":np.zeros_like(theta),
            "t":0
        }
    b1=parameters['β1']
    b2=parameters['β2']
    parameters['t'] = parameters['t'] + 1
    parameters['m'] = ((1 - b1) * xj) + (b1 * parameters['m'])
    parameters['v'] = ((1 - b2) * (xj ** 2)) + (b2 * parameters['v'])
    temp1 = 1 - (b1 ** parameters['t'])
    temp2 = 1 - (b2 ** parameters['t'])
    new_theta = theta - parameters['learn_rate'] * (parameters['m'] / temp1) / (np.sqrt(parameters['v'] / temp2) + parameters['e'])
    return new_theta,parameters

In [ ]:
class Train:
    """
    Lenet5 相当于传进来一个类，这个类保存了一次训练需要的信息
    data
    epoch 所有样本都跑过一遍所需次数
    func 选择哪个参数优化函数
    parameter
    decrease 学习率下降数
    batchSize 每次只使用数据集中的部分样本
    exp
    """
    def __init__(self, Lenet5, data, epoch, func, parameter, decrease, batchSize, exp):
        self.model = Lenet5
        self.train1 = data["train1"] #在传进来data前也得注意
        self.train2 = data["train2"]
        self.label1 = data["label1"]
        self.label2 = data["label2"]
        self.func = globals()[func] #让他指向了指定的那个函数
        self.learn_rate_decay = decrease #学习率下降
        self.batchSize = batchSize #一次使用的样本数
        self.exp = exp #不知道干啥用的 完全没用到
        self.epoch = epoch

        # 保存最高准确率以及最佳参数
        self.bestAccurate = 0
        self.bestPara = {}

        # 为了绘图
        self.losses = []
        self.TrainAccurates = []
        self.ProofAccurates = []

        # 按照他的意思这个东西是深度拷贝
        self.funcParameter = {}
        for i in self.moodel.params:
            d = {k: v for k,v in parameter.items()}
            self.funcParameter[i] = d

    # 直接返回向量
    def calAccuracy(self,train,label,batchSize):
        # 需要做几轮运算
        myRound = math.ceil(train.shape[0] / batchSize)
        # 最终的预测结果
        prediction = []
        for i in range(myRound):
            start = i*batchSize
            end = (i+1)*batchSize
            # 这个不太懂loss算了个什么
            # 为什么只用给train就行了，还有就是train是什么，最后数组保存的是什么
            # 实际loss返回两个值
            acc = self.model.loss(train[start:end]) #计算损失
            p = np.argmax(acc,axis=1) #找出最大值
            prediction.append(p)
        np.hstack(prediction) #横向拼接向量

        return prediction

    def calAccuracy2(self,train,label,batchSize):
        prediction = self.calAccuracy(train, label, batchSize)
        return np.mean(prediction == label)

    def myTrain(self):
        # 为了绘图 把这几个先初始化为空
        self.losses = []
        self.TrainAccurates = []
        self.ProofAccurates = []
        # 按照不同的轮以对应的batchsize训练
        for i in range(self.epoch):
            for j in range(math.ceil(self.train1.shape[0] / self.batchSize)):
                start = j * self.batchSize
                end = (j+1) * self.batchSize
                arr = np.arange(start,end)
                batch_train = self.train1[arr]
                batch_label = self.label1[arr]
                # 我觉得在算loss的时候就更新了model的params
                loss,res = self.model.loss(batch_train,batch_label)
                self.losses.append(loss)
                for k,v in self.model.params.items():
                    self.model.params[k],self.funcParameter[k] = self.func(v,res[k],self.funcParameter[k])
                print('第 %d 轮第 %d 个batch中loss为 %f' % (i+1,j+1,loss))

            self.TrainAccurates.append(self.calAccuracy2(self.train1,self.train2,100))
            self.ProofAccurates.append(self.calAccuracy2(self.label1,self.label2,100))

            if self.calAccuracy(self.label1,self.label2,100) > self.bestAccurate :
                self.bestAccurate = self.calAccuracy(self.label1,self.label2,100)
                self.bestPara.clear() #更新自己参数里的最佳参数
                for k,v in self.model.params.items():
                    self.bestPara[k] = v.copy()

            if i+1 == self.epoch:
                filename = "TrainingResult"
                with open(filename,'wb') as f:
                    pickle.dump(self.model,f)
                    print("成功保存模型")

            if i > 10:
                for q in self.funcParameter:
                    self.funcParameter[q]['learn_rate'] *= self.learn_rate_decay

        # 我觉得这里最后一轮出最佳的话就保存不了了啊
        self.model.params = self.bestPara

In [ ]:
# 卷积层前向传播的过程
def convolution_spread_forward(train, convolution_entity, bias, para):
    pic_num = train.shape[0]
    channel = train.shape[1]
    height = train[2]
    wide = train[3]
    convolution_num = convolution_entity[0]
    convolution_height = convolution_entity[2]
    convolution_wide = convolution_entity[3]
    step = para['step'] #这里代表着步长
    padding = para['pad'] # 填充部分的大小
    # 这里可以改成函数外提
    new_train = np.pad(train,((0,0),(0,0),(padding,padding),(padding,padding)))
    # 这里也可以函数外提
    loop_outside  = height + 2 * padding - convolution_height
    loop_outside  = loop_outside / step
    loop_outside  = math.floor(loop_outside) + 1
    loop_inside = wide + 2 * padding - convolution_wide
    loop_inside = loop_inside / step
    loop_inside = math.floor(loop_inside) + 1
    weight = np.zeros(shape=((pic_num,convolution_num,loop_outside,loop_inside)),dtype=float)
    for i in range(loop_outside):
        for j in range(loop_inside):
            temp = []
            # 这个矩阵转换必须外提做函数
            temp = new_train[:,:,step*i:step*i+convolution_height,step*j:step*j+convolution_wide]
            temp.reshape(pic_num,1,channel,convolution_height,convolution_wide)
            new_convolution = convolution_entity.reshape(1,convolution_num,channel,convolution_height,convolution_wide)
            # 这个也可以进行函数外提
            weight = np.sum(temp * new_convolution,axis=(-3,-2,-1))
            weight += bias
    # 这个也得封装
    package = (train,convolution_entity,bias,para)
    return weight,package

In [ ]:
def convolution_spread_backward(derivation,pack):
    num = derivation[0]
    convolution_num = derivation[1]
    loop_outside = derivation[2]
    loop_inside = derivation[3]
    train = pack[0]
    height = train[2]
    wide = train[3]
    convolution = pack[1]
    derivation_convolution = np.zeros_like(convolution)
    channel =convolution[1]
    convolution_height = convolution[2]
    convolution_wide = convolution[3]
    bias = pack[2]
    derivation_bias = np.zeros_like(bias)
    para = pack[3]
    padding = para['pad']
    step = para['step']
    # 这个可以封装成一个比较复杂的函数
    new_train = np.pad(train,((0,0),(0,0),(padding,padding),(padding,padding)))
    derivation_train = np.zeros_like(new_train)
    for i in range(loop_outside):
        for j in range(loop_inside):
            #都可以外提成函数
            temp = derivation[:,:,i,j]
            temp.reshape((num,1,1,1,convolution_num))
            temp2 = convolution.transpose((1,2,3,0))
            temp2.reshape((1,channel,convolution_height,convolution_wide,convolution_num))
            temp_sum = np.sum(temp*temp2,axis=-1)
            derivation_train[:,:,step*i:step*i+convolution_height,step*j:step*j+convolution_wide] +=temp_sum
            temp3 = derivation[:,:,i,j].T
            temp3 = temp3.reshape((convolution_num,1,1,1,num))
            temp4 = new_train[:,:,step*i:step*i+convolution_height,step*j:step*j+convolution_wide]
            # 外提做函数
            temp4.transpose(1,2,3,0)
            temp_sum = np.sum(temp3*temp4,axis=-1)
            derivation_convolution += temp_sum
            temp_sum = np.sum(derivation[:,:,i,j],axis=0)
            derivation_bias += temp_sum

    # 这个必须外提
    derivation_weight = []
    derivation_weight = derivation_train[:,:,padding:padding+height,padding:padding+wide]
    return derivation_weight,derivation_convolution,derivation_bias